In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'sklearn'

# Linear Regression

In [ ]:
## This is for Linear Regression

# Polynomial Regression

In [ ]:
## This is for Polynomial Regression